In [ ]:
#working code

import requests
from bs4 import BeautifulSoup
import pyodbc
from datetime import datetime

# URL of the page containing vegetable prices
url = 'https://vegetablemarketprice.com/market/karnataka/today'

# Send a GET request to fetch the HTML content
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the vegetable prices
table = soup.find('tbody', id='todayVegetableTableContainer')

# Initialize an empty list to store the extracted data
vegetable_prices = []

# List of vegetables to scrape
vegetables_to_scrape = ['Tomato', 'Onion Big', 'Potato', 'Cauliflower', 'Carrot', 'Brinjal', 'Ginger', 'Garlic', 'Radish', 'Ladies Finger']

# Loop through each row in the table
for row in table.find_all('tr', class_='todayVegetableTableRows'):
    cols = row.find_all('td')
    vegetable = cols[1].text.strip()
    
    # Check if the vegetable is in the list of vegetables to scrape and is not "Sweet Potato"
    if any(veg in vegetable for veg in vegetables_to_scrape) and 'Sweet Potato' not in vegetable:
        retail_price_range = cols[3].text.strip()
        unit = cols[5].text.strip()

        # Calculate the average retail price
        if '-' in retail_price_range:
            low, high = retail_price_range.split('-')
            avg_retail_price = (float(low.strip('₹ ')) + float(high.strip('₹ '))) / 2
        else:
            avg_retail_price = float(retail_price_range.strip('₹ '))

        # Append the data to the list
        vegetable_prices.append({
            'vegetable': vegetable,
            'avg_retail_price': f'{avg_retail_price:.2f}',  # Remove ₹ symbol for numeric storage
            'unit': unit
        })

# Print the extracted data
for price in vegetable_prices:
    print(f"Vegetable: {price['vegetable']}")
    print(f"Average Retail Price: {price['avg_retail_price']}")
    print(f"Unit: {price['unit']}")
    print('-' * 40)

# Replace with your actual Oracle database DSN, user, and password
dsn = 'mydb'
user = 'system'
password = 'system'

# Create a connection to the database
conn = pyodbc.connect(f'DSN={dsn};UID={user};PWD={password}')

# Create a cursor object
cursor = conn.cursor()

# Get the current date in YYYY-MM-DD format
date = datetime.now().strftime('%Y-%m-%d')

# Insert data into the database
for price in vegetable_prices:
        cursor.execute('''INSERT INTO liveprice (item, dte, price) 
                          VALUES (?, TO_DATE(?, 'YYYY-MM-DD'), ?)''',(price['vegetable'], date, price['avg_retail_price']))
conn.commit()


conn.close()


print("inserted todays data")




Vegetable: Onion Big
Average Retail Price: 42.00
Unit: 1kg
----------------------------------------
Vegetable: Tomato
Average Retail Price: 31.50
Unit: 1kg
----------------------------------------
Vegetable: Potato
Average Retail Price: 43.50
Unit: 1kg
----------------------------------------
Vegetable: Carrot
Average Retail Price: 48.50
Unit: 1kg
----------------------------------------
Vegetable: Cauliflower
Average Retail Price: 35.00
Unit: 1kg
----------------------------------------
Vegetable: Brinjal
Average Retail Price: 43.50
Unit: 1kg
----------------------------------------
Vegetable: Brinjal (Big)
Average Retail Price: 47.50
Unit: 1kg
----------------------------------------
Vegetable: Garlic
Average Retail Price: 380.00
Unit: 1kg
----------------------------------------
Vegetable: Ginger
Average Retail Price: 71.50
Unit: 1kg
----------------------------------------
Vegetable: Ladies Finger
Average Retail Price: 47.50
Unit: 1kg
----------------------------------------
Vegeta

In [ ]:
#working code

import pyodbc
from datetime import datetime

# Replace with your actual Oracle database DSN, user, and password
dsn = 'mydb'
user = 'system'
password = 'system'

# Create a connection to the database
conn = pyodbc.connect(f'DSN={dsn};UID={user};PWD={password}')

# Create a cursor object
cursor = conn.cursor()

# Take user input for the date
dt_to_check = datetime.now().strftime('%d-%m-%y')

# Define the SQL query with a parameter placeholder for the date
sql_query = """
SELECT item, price, round(perc_diff) 
FROM (
    SELECT 
        price,
        dte,
        item,
        AVG(price) OVER (PARTITION BY item ORDER BY dte ROWS BETWEEN 9 PRECEDING AND CURRENT ROW) AS running_avg, 
        ((price - (AVG(price) OVER (PARTITION BY item ORDER BY dte ROWS BETWEEN 9 PRECEDING AND CURRENT ROW))) / 
        (AVG(price) OVER (PARTITION BY item ORDER BY dte ROWS BETWEEN 9 PRECEDING AND CURRENT ROW))) * 100 AS perc_diff
    FROM liveprice
)
WHERE dte = TO_DATE(?, 'dd-mm-yy')
ORDER BY perc_diff
"""

# Execute the query with the user-provided date as a parameter
cursor.execute(sql_query, dt_to_check)

# Fetch all rows from the query
rows = cursor.fetchall()

# Extract the items from the rows and print them
result = [dict(zip([column[0] for column in cursor.description], row)) for row in rows]
i = 1
for item in result:
    print('serial no:{}'.format(i))
    i = i +1
    print(item)

# Close the cursor and connection
cursor.close()
conn.close()



serial no:1
{'ITEM': 'Onion Big', 'PRICE': Decimal('42'), 'ROUND(PERC_DIFF)': -36.0}
serial no:2
{'ITEM': 'Potato', 'PRICE': Decimal('43.5'), 'ROUND(PERC_DIFF)': -22.0}
serial no:3
{'ITEM': 'Radish', 'PRICE': Decimal('47.5'), 'ROUND(PERC_DIFF)': -9.0}
serial no:4
{'ITEM': 'Garlic', 'PRICE': Decimal('380'), 'ROUND(PERC_DIFF)': -8.0}
serial no:5
{'ITEM': 'Carrot', 'PRICE': Decimal('48.5'), 'ROUND(PERC_DIFF)': -8.0}
serial no:6
{'ITEM': 'Cauliflower', 'PRICE': Decimal('35'), 'ROUND(PERC_DIFF)': -3.0}
serial no:7
{'ITEM': 'Ginger', 'PRICE': Decimal('71.5'), 'ROUND(PERC_DIFF)': -3.0}
serial no:8
{'ITEM': 'Ladies Finger', 'PRICE': Decimal('47.5'), 'ROUND(PERC_DIFF)': -1.0}
serial no:9
{'ITEM': 'Brinjal', 'PRICE': Decimal('43.5'), 'ROUND(PERC_DIFF)': 0.0}
serial no:10
{'ITEM': 'Brinjal (Big)', 'PRICE': Decimal('47.5'), 'ROUND(PERC_DIFF)': 0.0}
serial no:11
{'ITEM': 'Tomato', 'PRICE': Decimal('31.5'), 'ROUND(PERC_DIFF)': 8.0}


In [ ]:
from decimal import Decimal

# Input data
data = result

# Convert to desired format
converted_data = [
    {
        "item": item["ITEM"],
        "price": float(item["PRICE"]),  # Convert Decimal to float
        "perc_diff": item["ROUND(PERC_DIFF)"]
    }
    for item in data
]

# Print the result
print(converted_data)


[{'item': 'Onion Big', 'price': 42.0, 'perc_diff': -36.0}, {'item': 'Potato', 'price': 43.5, 'perc_diff': -22.0}, {'item': 'Radish', 'price': 47.5, 'perc_diff': -9.0}, {'item': 'Garlic', 'price': 380.0, 'perc_diff': -8.0}, {'item': 'Carrot', 'price': 48.5, 'perc_diff': -8.0}, {'item': 'Cauliflower', 'price': 35.0, 'perc_diff': -3.0}, {'item': 'Ginger', 'price': 71.5, 'perc_diff': -3.0}, {'item': 'Ladies Finger', 'price': 47.5, 'perc_diff': -1.0}, {'item': 'Brinjal', 'price': 43.5, 'perc_diff': 0.0}, {'item': 'Brinjal (Big)', 'price': 47.5, 'perc_diff': 0.0}, {'item': 'Tomato', 'price': 31.5, 'perc_diff': 8.0}]


In [ ]:
from flask import Flask, jsonify, render_template

app = Flask(__name__)

fruits_data = [
    {"serial_no": 1, "name": "Apple", "price": "$3"},
    {"serial_no": 2, "name": "Banana", "price": "$1"},
    {"serial_no": 3, "name": "Orange", "price": "$2.5"}
]

vegetables_data = converted_data

@app.route('/')
def home():
    return render_template("index.html")

@app.route("/api/fruits")
def get_fruits():
    return jsonify(fruits_data)

@app.route("/api/vegetables")
def get_vegetables():
    return jsonify(vegetables_data)




if __name__ == '__main__':
    app.run(debug=True,host='0.0.0.0', port=5000, use_reloader=False)

Traceback (most recent call last):
  File "c:\Users\keerthana\.vscode\extensions\ms-python.python-2024.22.1-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 3, in <module>
  File "C:\Users\keerthana\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 231, in __init__
    super().__init__(
  File "C:\Users\keerthana\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\sansio\app.py", line 295, in __init__
    super().__init__(
  File "C:\Users\keerthana\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\sansio\scaffold.py", line 96, in __init__
    root_path = get_root_path(self.import_name)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\keerthana\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\helpers.py", line 601, in get_root_path
    raise RuntimeError(
Runti